# 数据准备
## 数据集下载

我们选用 Datawhale 一些经典开源课程作为示例，具体包括：

* [《机器学习公式详解》PDF版本](https://github.com/datawhalechina/pumpkin-book/releases)
* [《面向开发者的LLM入门教程、第一部分Prompt Engineering》md版本](https://github.com/datawhalechina/llm-cookbook)  

In [1]:
!ls ../data

LLM-v1.0.0.pdf            LLM-v1.0.0_md.zip         pumpkin_book.pdf
LLM-v1.0.0_latex.zip      LLM-v1.0.0_md_dollar.zip


# 文本数据解析

In [2]:
from langchain.document_loaders.pdf import PyMuPDFLoader
import pandas as pd

# 创建一个 PyMuPDFLoader Class 实例，输入为待加载的 pdf 文档路径
loader = PyMuPDFLoader("../data/pumpkin_book.pdf")

# 调用 PyMuPDFLoader Class 的函数 load 对 pdf 文件进行加载
pdf_pages = loader.load()

In [3]:
pdf_page = pdf_pages[1]
print(f"每一个元素的类型：{type(pdf_page)}.", 
    f"该文档的描述性数据：{pdf_page.metadata}", 
    f"查看该文档的内容:\n{pdf_page.page_content}", 
    sep="\n------\n")

每一个元素的类型：<class 'langchain_core.documents.base.Document'>.
------
该文档的描述性数据：{'source': '../data/pumpkin_book.pdf', 'file_path': '../data/pumpkin_book.pdf', 'page': 1, 'total_pages': 196, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'LaTeX with hyperref', 'producer': 'xdvipdfmx (20200315)', 'creationDate': "D:20231117152045-00'00'", 'modDate': '', 'trapped': ''}
------
查看该文档的内容:
前言
“周志华老师的《机器学习》
（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读
者通过西瓜书对机器学习有所了解, 所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推
导细节的读者来说可能“不太友好”
，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充
具体的推导细节。
”
读到这里，大家可能会疑问为啥前面这段话加了引号，因为这只是我们最初的遐想，后来我们了解到，周
老师之所以省去这些推导细节的真实原因是，他本尊认为“理工科数学基础扎实点的大二下学生应该对西瓜书
中的推导细节无困难吧，要点在书里都有了，略去的细节应能脑补或做练习”
。所以...... 本南瓜书只能算是我
等数学渣渣在自学的时候记下来的笔记，希望能够帮助大家都成为一名合格的“理工科数学基础扎实点的大二
下学生”
。
使用说明
• 南瓜书的所有内容都是以西瓜书的内容为前置知识进行表述的，所以南瓜书的最佳使用方法是以西瓜书
为主线，遇到自己推导不出来或者看不懂的公式时再来查阅南瓜书；
• 对于初学机器学习的小白，西瓜书第1 章和第2 章的公式强烈不建议深究，简单过一下即可，等你学得
有点飘的时候再回来啃都来得及；
• 每个公式的解析和推导我们都力(zhi) 争(neng) 以本科数学基础的视角进行讲解，所以超纲的数学知识

## 使用[pdfdeal](https://github.com/Menghuan1918/pdfdeal?tab=readme-ov-file)库进行pdf文件处理(待优化)

In [4]:
# from pdfdeal.doc2x import Doc2X
# from dotenv import load_dotenv, find_dotenv
# find_dotenv()
# load_dotenv()
# import os 
# Client = Doc2X()
# file_type = 'pdf'
# path = '../data/'
# def gen_folder_list(path,file_type):

#     for root, dirs, files in os.walk(path):
#         # print(root,dirs,files)
#         pdf_list = []
#         for file in files:
#             if file.endswith(f'.{file_type}'):
#                 # print(os.path.join(root,file))
#                 pdf_list.append(os.path.join(root,file))
#     return pdf_list
# filelist = gen_folder_list(path,file_type)
# # This is a built-in function for generating the folder under the path of all the pdf, you can give any list of the form of the path of the pdf
# Client.pdfdeal(filelist)




## 数据清洗

In [5]:
import re
pattern = re.compile(r'[^\u4e00-\u9fff](\n)[^\u4e00-\u9fff]', re.DOTALL)
def clearn_page_content(pdf_page):

    pdf_page.page_content = re.sub(pattern, lambda match: match.group(0).replace('\n', ''), pdf_page.page_content)
    pdf_page.page_content = pdf_page.page_content.replace('•', '')
    pdf_page.page_content = pdf_page.page_content.replace(' ', '')
    # print(pdf_page.page_content)
    return pdf_page
clearn_page_content(pdf_page)

Document(page_content='前言\n“周志华老师的《机器学习》（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读\n者通过西瓜书对机器学习有所了解,所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推\n导细节的读者来说可能“不太友好”，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充\n具体的推导细节。”\n读到这里，大家可能会疑问为啥前面这段话加了引号，因为这只是我们最初的遐想，后来我们了解到，周\n老师之所以省去这些推导细节的真实原因是，他本尊认为“理工科数学基础扎实点的大二下学生应该对西瓜书\n中的推导细节无困难吧，要点在书里都有了，略去的细节应能脑补或做练习”。所以......本南瓜书只能算是我\n等数学渣渣在自学的时候记下来的笔记，希望能够帮助大家都成为一名合格的“理工科数学基础扎实点的大二\n下学生”。\n使用说明\n南瓜书的所有内容都是以西瓜书的内容为前置知识进行表述的，所以南瓜书的最佳使用方法是以西瓜书\n为主线，遇到自己推导不出来或者看不懂的公式时再来查阅南瓜书；对于初学机器学习的小白，西瓜书第1章和第2章的公式强烈不建议深究，简单过一下即可，等你学得\n有点飘的时候再回来啃都来得及；每个公式的解析和推导我们都力(zhi)争(neng)以本科数学基础的视角进行讲解，所以超纲的数学知识\n我们通常都会以附录和参考文献的形式给出，感兴趣的同学可以继续沿着我们给的资料进行深入学习；若南瓜书里没有你想要查阅的公式，\n或者你发现南瓜书哪个地方有错误，\n请毫不犹豫地去我们GitHub的\nIssues（地址：https://github.com/datawhalechina/pumpkin-book/issues）进行反馈，在对应版块\n提交你希望补充的公式编号或者勘误信息，我们通常会在24小时以内给您回复，超过24小时未回复的\n话可以微信联系我们（微信号：at-Sm1les）；\n配套视频教程：https://www.bilibili.com/video/BV1Mh411e7VU\n在线阅读地址：https://datawhalechina.github.io/pumpkin-book（仅供第1版）\n最新版PDF获取地址：https://github.com/datawhalechina

# 文本切分

In [6]:

from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
# 知识库中单段文本长度
CHUNK_SIZE = 500

# 知识库中相邻文本重合长度
OVERLAP_SIZE = 50

In [8]:
# 使用递归字符文本分割器
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=OVERLAP_SIZE
)
split_docs = text_splitter.split_documents(pdf_pages)
print(f"切分后的文件数量：{len(split_docs)}")

print(f"切分后的字符数（可以用来大致评估 token 数）：{sum([len(doc.page_content) for doc in split_docs])}")

切分后的文件数量：720
切分后的字符数（可以用来大致评估 token 数）：308791


# 模型搭建

In [9]:
from dotenv import load_dotenv, find_dotenv
import os

_ = load_dotenv(find_dotenv())    # read local .env file
zhipuai_api_key = os.environ['ZHIPUAI_API_KEY']

In [10]:
from zhipuai_llm import ZhipuAILLM

In [11]:
# llm = ZhipuAILLM(temperature=0,api_key=os.environ['ZHIPUAI_API_KEY'])
llm = ZhipuAILLM(
    model = 'glm-4',
    max_tokens = 256,
    temperature = 0.8,
    api_key=os.environ['ZHIPUAI_API_KEY']
)

In [12]:
llm('你好')

/opt/anaconda3/envs/llm/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'你好👋！我是人工智能助手智谱清言，可以叫我小智🤖，很高兴见到你，欢迎问我任何问题。'

# Milvus向量库

In [13]:
from pymilvus import utility
from pymilvus import connections
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection
import csv
import time
from tqdm import tqdm 
from sentence_transformers import SentenceTransformer



In [14]:
COLLECTION_NAME = 'rag_db'  # Collection name
DIMENSION = 768  # Embeddings size
COUNT = 1000  # Number of vectors to insert
MILVUS_HOST = 'localhost'
MILVUS_PORT = '19530' # Inference Arguments
BATCH_SIZE = 128
MAX_LENGTH = 4096

In [15]:
connections.connect(host=MILVUS_HOST, port=MILVUS_PORT)

In [16]:
def document2df(pdf_pages):
    # 用于获取元数据 存储到向量数据库进行混合向量查询
    df_list = []
    for pdf_page in pdf_pages:
        pdf_page = clearn_page_content(pdf_page)
        pdf_page_dict = pdf_page.dict().get('metadata')
        pdf_page_dict.update({'page_content':pdf_page.dict().get('page_content')})
        df = pd.json_normalize(pdf_page_dict)
        df_list.append(df)
    df = pd.concat(df_list, ignore_index=True).reset_index()  
    return df
df = document2df(pdf_pages)

In [55]:
df.head()

,index,source,file_path,page,total_pages,format,title,author,subject,keywords,creator,producer,creationDate,modDate,trapped,page_content
0,0,../data/pumpkin_book.pdf,../data/pumpkin_book.pdf,0,196,PDF 1.5,,,,,LaTeX with hyperref,xdvipdfmx (20200315),D:20231117152045-00'00',,,本:2.0.0\n发布日期:2023.11\n南⽠书\nPUMPKINBOOK\n谢\t...
1,1,../data/pumpkin_book.pdf,../data/pumpkin_book.pdf,1,196,PDF 1.5,,,,,LaTeX with hyperref,xdvipdfmx (20200315),D:20231117152045-00'00',,,前言\n“周志华老师的《机器学习》（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽...
2,2,../data/pumpkin_book.pdf,../data/pumpkin_book.pdf,2,196,PDF 1.5,,,,,LaTeX with hyperref,xdvipdfmx (20200315),D:20231117152045-00'00',,,→_→\n欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解第2版》←_←\n目录\n第1...
3,3,../data/pumpkin_book.pdf,../data/pumpkin_book.pdf,3,196,PDF 1.5,,,,,LaTeX with hyperref,xdvipdfmx (20200315),D:20231117152045-00'00',,,→_→\n欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解第2版》←_←3.3.1\n式...
4,4,../data/pumpkin_book.pdf,../data/pumpkin_book.pdf,4,196,PDF 1.5,,,,,LaTeX with hyperref,xdvipdfmx (20200315),D:20231117152045-00'00',,,→_→\n欢迎去各大电商平台选购纸质版南瓜书《机器学习公式详解第2版》←_←5.5\n其他常...


In [18]:
def delete_collection(COLLECTION_NAME):
    if utility.has_collection(COLLECTION_NAME):
        utility.drop_collection(COLLECTION_NAME)
delete_collection(COLLECTION_NAME) 

In [19]:

def create_collection(collection_name):
 
    # 主键
    field_id = FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True)
    # 向量检索的field
    field_title = FieldSchema(name='page', dtype=DataType.INT64,  description ='page', max_length=MAX_LENGTH )
    field_origin = FieldSchema(name='page_content', dtype=DataType.VARCHAR, description ='page_content' , max_length=8192 )

    field_title_embedding = FieldSchema(name='page_content_embedding', dtype=DataType.FLOAT_VECTOR, dim=DIMENSION,description ='page_content' )
    # field_plot_embedding = FieldSchema(name='plot_embedding', dtype=DataType.FLOAT_VECTOR,dim=64,description ='Plot' )
    schema = CollectionSchema(fields=[field_id, 
                                      field_title, 
                                      field_origin,
                              
                                      field_title_embedding,
                                      # field_plot_embedding
                                     ], description="page_content_collection")

    collection = Collection(name=collection_name, schema=schema)
   

    return collection
  

collection = create_collection(COLLECTION_NAME)
### 为集合创建IVF_FLAT索引
def create_index_collection(collection):

    
    index_params = {
        'metric_type':'L2',
        'index_type':"IVF_FLAT",
        'params':{'nlist': 1536}
    }
    collection.create_index(field_name="page_content_embedding", index_params=index_params)
    collection.load()
create_index_collection(collection)

## 读取模型

In [20]:

transformer = SentenceTransformer('/Users/heitao/models/AI-ModelScope/bge-base-zh-v1-5', )
# transformer.encode(x)
 

## 为collection创建分区

In [21]:
def create_partition(collection,partition_name):
    """
    为collection创建分区
    :param collection:
    :return:
    """
    partition = collection.create_partition(partition_name)
    print(collection.partitions)
    print(collection.has_partition(partition_name))
    
create_partition(collection,partition_name = 'partition_test')  

[{"name":"_default","collection_name":"rag_db","description":""}, {"name":"partition_test","collection_name":"rag_db","description":""}]
True


## 插入数据

In [22]:
def insert_data_collection(df ):
    
    field_page = df['page'].to_list()
    field_page_content = df['page_content'].to_list()

    sentences = df['page_content'].to_list()
    embeddings = transformer.encode(sentences)
    ins = [ field_page,field_page_content,embeddings]
    collection.insert(ins)
    collection.flush()
insert_data_collection(df)

In [23]:
def embedding_query(search_terms):
    embeds = transformer.encode(search_terms) 
    return [x for x in embeds]
# Search for titles that closest match these phrases.



In [24]:
def get_result(search_terms, top_k=3):
    
    search_data = embedding_query(search_terms)
    res = collection.search(
        data=search_data,  # Embeded search value
        anns_field="page_content_embedding",  # Search across embeddings
        param={
                # "nprobe": 128,
                # "metric_type": "L2",
                # "offset": 10,
                # "limit": 10,
                        },
        limit = top_k,  # Limit to top_k results per search
        output_fields=['page_content']  # Include title field in result
    )
    result = []
    for hits_i, hits in enumerate(res):
        print('Title:', search_terms[hits_i])
        # print('Search Time:', end-start)
        # print('Results:')
        for hit in hits:
            # print( hit.entity.get('page_content'), '----', hit.distance)
            result.append(hit.entity.get('page_content'))
    return result


# 构建检索问答链

## 自己定义一个

In [25]:
def rag(question):
    search_terms = [question]
    result = get_result(search_terms)
    template = """使用以下上下文来回答最后的问题。如果所给的上下文中没有提到相关的答案，就说你不知道，不要试图编造答
    案。最多使用三句话。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问！”。
    上下文内容：{}
    问题: {}
    """.format('\n'.join(result),search_terms[0])

    answer = llm(template)
    print(answer)
    return answer
question = '孙悟空的师傅是谁'
answer = rag(question)

Title: 孙悟空的师傅是谁
我不知道孙悟空的师傅是谁，因为上下文中没有提到相关信息。谢谢你的提问！


## 使用 RetrievalQA

In [ ]:
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
import getpass
import os

from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores.milvus import Milvus
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_text_splitters.character import CharacterTextSplitter
embedding_model = HuggingFaceEmbeddings(model_name='/Users/heitao/models/AI-ModelScope/bge-base-zh-v1-5')

loader = PyPDFLoader("../data/pumpkin_book.pdf")
data = loader.load()

# Split docs
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=100)
docs = text_splitter.split_documents(data)

# Insert the documents in Milvus Vector Store
vector_db = Milvus.from_documents(
    docs,
    embedding_model,
    collection_name='test',
    connection_args={"host": MILVUS_HOST, "port": MILVUS_PORT},
    )


template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。最多使用三句话。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vector_db.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})
question_1 = "什么是南瓜书？"
question_2 = "王阳明是谁？"

result = qa_chain({"query": question_1})
print("大模型+知识库后回答 question_1 的结果：")
print(result["result"])
 

# 部署知识库助手 

In [26]:
import streamlit as st
from langchain_openai import ChatOpenAI
st.title('🦜🔗 动手学大模型应用开发')
zhipuai_api_key = st.sidebar.text_input('ZhiPu API Key', type='password')
def generate_response(input_text):
    llm = ChatOpenAI(temperature=0.7, openai_api_key=zhipuai_api_key)
    st.info(llm(input_text))
with st.form('my_form'):
    text = st.text_area('Enter text:', 'What are the three key pieces of advice for learning how to code?')
    submitted = st.form_submit_button('Submit')
    if not zhipuai_api_key.startswith('sk-'):
        st.warning('Please enter your OpenAI API key!', icon='⚠')
    if submitted and zhipuai_api_key.startswith('sk-'):
        generate_response(text)

2024-06-27 11:41:49.968 
  command:

    streamlit run /opt/anaconda3/envs/llm/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]


In [ ]:
import streamlit as st
from langchain_openai import ChatOpenAI
import os
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
import sys
sys.path.append("../C3 搭建知识库") # 将父目录放入系统路径中
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
# from zhipuai_embedding import ZhipuAIEmbeddings
from langchain.vectorstores.chroma import Chroma
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters.character import CharacterTextSplitter
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())    # read local .env file
from zhipuai_llm import ZhipuAILLM
from langchain.vectorstores.milvus import Milvus

#export OPENAI_API_KEY=
#os.environ["OPENAI_API_BASE"] = 'https://api.chatgptid.net/v1'
zhipuai_api_key = os.environ['ZHIPUAI_API_KEY']
embedding = HuggingFaceEmbeddings(model_name='/Users/heitao/models/AI-ModelScope/bge-base-zh-v1-5')
loader = PyPDFLoader("../data/pumpkin_book.pdf")
data = loader.load()
# llm = ZhipuAILLM(temperature=0,api_key=os.environ['ZHIPUAI_API_KEY'])
llm = ZhipuAILLM(
    model = 'glm-4',
    max_tokens = 256,
    temperature = 0.8,
    api_key=os.environ['ZHIPUAI_API_KEY']
)
# Split docs
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=100)
docs = text_splitter.split_documents(data)
def generate_response(input_text, openai_api_key,llm = llm):
    # llm = ChatOpenAI(temperature=0.7, openai_api_key=openai_api_key)
    output = llm.invoke(input_text)
    output_parser = StrOutputParser()
    output = output_parser.invoke(output)
    #st.info(output)
    return output

def get_vectordb(docs = docs,embedding = embedding):
    MILVUS_HOST = 'localhost'
    MILVUS_PORT = '19530' # Inference Arguments
    # 定义 Embeddings
    # embedding = ZhipuAIEmbeddings()
    
    # 向量数据库持久化路径

    # persist_directory = './chroma'
    # 加载数据库
    # vectordb = Chroma(
    #     persist_directory=persist_directory,  # 允许我们将persist_directory目录保存到磁盘上
    #     embedding_function=embedding
    # )
    vectordb = Milvus.from_documents(
    docs,
    embedding,
    collection_name='test',
    connection_args={"host": MILVUS_HOST, "port": MILVUS_PORT},
    )
    return vectordb

#带有历史记录的问答链
def get_chat_qa_chain(question:str,openai_api_key:str,llm = llm):
    vectordb = get_vectordb()
    # llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature = 0,openai_api_key = openai_api_key)

    memory = ConversationBufferMemory(
        memory_key="chat_history",  # 与 prompt 的输入变量保持一致。
        return_messages=True  # 将以消息列表的形式返回聊天记录，而不是单个字符串
    )
    retriever=vectordb.as_retriever()
    qa = ConversationalRetrievalChain.from_llm(
        llm,
        retriever=retriever,
        memory=memory
    )
    result = qa({"question": question})
    return result['answer']

#不带历史记录的问答链
def get_qa_chain(question:str,openai_api_key:str,llm = llm):
    vectordb = get_vectordb()
    # llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature = 0,openai_api_key = openai_api_key)
    template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
        案。最多使用三句话。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问！”。
        {context}
        问题: {question}
        """
    QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template)
    qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})
    result = qa_chain({"query": question})
    return result["result"]


# Streamlit 应用程序界面
def main():
    st.title('🦜🔗 动手学大模型应用开发')
    openai_api_key = st.sidebar.text_input('暗号', type='password')

    # 添加一个选择按钮来选择不同的模型
    #selected_method = st.sidebar.selectbox("选择模式", ["qa_chain", "chat_qa_chain", "None"])
    selected_method = st.radio(
        "你想选择哪种模式进行对话？",
        ["None", "qa_chain", "chat_qa_chain"],
        captions = ["不使用检索问答的普通模式", "不带历史记录的检索问答模式", "带历史记录的检索问答模式"])

    # 用于跟踪对话历史
    if 'messages' not in st.session_state:
        st.session_state.messages = []

    messages = st.container(height=300)
    if prompt := st.chat_input("Say something"):
        # 将用户输入添加到对话历史中
        st.session_state.messages.append({"role": "user", "text": prompt})

        if selected_method == "None":
            # 调用 respond 函数获取回答
            answer = generate_response(prompt, openai_api_key)
        elif selected_method == "qa_chain":
            answer = get_qa_chain(prompt,openai_api_key)
        elif selected_method == "chat_qa_chain":
            answer = get_chat_qa_chain(prompt,openai_api_key)

        # 检查回答是否为 None
        if answer is not None:
            # 将LLM的回答添加到对话历史中
            st.session_state.messages.append({"role": "assistant", "text": answer})

        # 显示整个对话历史
        for message in st.session_state.messages:
            if message["role"] == "user":
                messages.chat_message("user").write(message["text"])
            elif message["role"] == "assistant":
                messages.chat_message("assistant").write(message["text"])   


if __name__ == "__main__":
    main()

### 优化点
1. Chroma本地向量库改为 Milvus 线上向量库
2. 元数据一同存储到Milvus，支持混合检索
3. 线上的 embedding 模型改为本地的embedding
